# Предсказание на 15 минут. Тестирование на реальных данных.

## Общая подготовка

Загрузим функции из предыдущего ноутбука.

### Необходимые библиотеки

In [1]:
!pip install catboost
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=b6616cac50b4f9cde937ac769f358eec4b9d6cc9eb6a76b9af4d49ebb338b8cb
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8

In [2]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import scipy.stats as sps
from scipy.ndimage.interpolation import shift
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

# Прикручивает диск к коллабу
from google.colab import drive
drive.mount("/content/drive")

# Модели, метрики
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

<ipython-input-2-8ce7d237dfc8>:5: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Загрузка исторических предобработанных хданных

In [ ]:
preprocessed_data_w2v = pd.read_csv('/content/drive/My Drive/data/preprocessed_data_w2v.csv') 
preprocessed_data_trans_emb = pd.read_csv('/content/drive/My Drive/data/preprocessed_data_trans_emb.csv') 
preprocessed_data = pd.read_csv('/content/drive/My Drive/data/preprocessed_data.csv') 

### Функции

In [5]:
def volatility_estimation(price, window_size):
  """
  Функция для оценки волатильности по периоду равному window_size

  param: price -- массив с ценой актива
  param: window_size -- Размер окна 
  """

  vol = []
  # Рассчет лог-доходности, не берем первый эл-т, т.к. он NaN
  log_return = np.log(price).diff()[1:]
  # Идем по данным с окном window_size, оцениваем дисперсию
  for i in range(len(log_return)-window_size):
        est = np.array(log_return.iloc[i:i+window_size]).std(ddof=1)
        vol.append(est)
  # Для сравнения размера дозаполним массив nan'ами.
  for _ in range(window_size+1):
    vol.append(np.nan)

  return np.array(vol)*np.sqrt(window_size)

def split_and_calc_target(data, window_sz, price_col = 'close'):
  """
    Функция для разделения на трейн  и тест исходя из горизонта прогнозирования
    
    param: data -- данные
    param: window_sz -- горузонт предсказания
  """
  X_train, X_test =  data[:-2*(window_sz)], data[-2*(window_sz):]
  y_train, y_test = volatility_estimation(X_train[price_col], window_sz), volatility_estimation(X_test[price_col], window_sz)

  X_train, X_test = X_train[X_train.columns.difference([price_col])], X_test[X_test.columns.difference([price_col])]
  X_train, X_test = X_train[:-(window_sz+1)], X_test[:-(window_sz+1)]
  y_train, y_test = y_train[:-(window_sz+1)], y_test[:-(window_sz+1)]
  return X_train, y_train, X_test, y_test

In [6]:
def create_date_features(data):
    """Создает фичи из даты"""

    assert "date" in data.columns,  "No date column"

    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d %X")

    data["dayofweek"] = [date.dayofweek for date in data["date"]]
    data["quarter"] = [date.quarter for date in data["date"]]
    data["month"] = [date.month for date in data["date"]]
    data["year"] = [date.year for date in data["date"]]
    data["dayofyear"] = [date.dayofyear for date in data["date"]]
    data["dayofmonth"] = [date.day for date in data["date"]]
    data["weekofyear"] = [date.weekofyear for date in data["date"]]
    return data[data.columns.difference(['date'])]

def generate_lag_features_train(dataset, target, shift_lst):
    """Функция создания признаков из дат исдвигов ряда для тестовых дат
    
    param: data -- Колонка дат
    param: target -- Колонка целевой переменной
    param: shift -- сдвиг # окон по 5 минут на сдвиг
    param: day_shift -- Сдвиг по дням
    param: mnth_shift -- сдвиг по месяцам
    """
    for lag in shift_lst:
      dataset[f'lag_{lag}'] = target.shift(lag)

    return dataset



def generate_lag_features_test(dataset_test, target_train, shift_lst):
    """Функция создания признаков из дат исдвигов ряда для тестовых дат
    
    param: data -- Колонка дат
    param: target -- Колонка целевой переменной
    param: shift -- сдвиг # окон по 5 минут на сдвиг
    param: day_shift -- Сдвиг по дням
    param: mnth_shift -- сдвиг по месяцам
    """
    test_sz = len(dataset_test)
    for lag in shift_lst:
      lag_feature = target_train.shift(lag)
      lag_feature = np.array(lag_feature[-test_sz:])    
      dataset_test[f'lag_{lag}'] = lag_feature
  

    return dataset_test

## Статистический тест

Для проведения статистического теста возьмем период длиной в 1 день. Исходя из прошлого ноутбука и выводов теперь будем прогнозировать на 15 минут вперед. Для прогнозирования будем использовать свежие данные. 

Реализуем это в коде.

In [8]:
# Загрузка данных
data = pd.read_csv('/content/drive/My Drive/data/preprocessed_test_table.csv')

# Колонка для оценки волатильности, период в день на тест
price_col = 'close'
X_train, X_test =  data[:-500], data[-500:]
y_train, y_test = volatility_estimation(X_train[price_col], 3), volatility_estimation(X_test[price_col], 3)
# Убираем колонку по которой проводился рассчет, чтобы не слить таргет
X_train, X_test = X_train[X_train.columns.difference([price_col])], X_test[X_test.columns.difference([price_col])]
X_train, X_test = X_train[:-(3+1)], X_test[:-(3+1)]
y_train, y_test = y_train[:-(3+1)], y_test[:-(3+1)]
# Фиксируем длину предсказания
WINDOW_SZ = 4
TEST_SZ = 4
# Фичи по дате, лаг фичи
X_train, X_test = create_date_features(X_train) , create_date_features(X_test)
lag_lst = [WINDOW_SZ+1, 2*WINDOW_SZ+1, 3*WINDOW_SZ+1, 5*WINDOW_SZ+1, 10*WINDOW_SZ+1]

X_test.reset_index(inplace = True, drop = True)
for lag in lag_lst:
  X_test[f'lag_{lag}'] = pd.DataFrame(y_test).shift(lag)
  # Для первых объектов теста тянем лаги из трейна
  for i in range(lag):
    X_test.at[i, f'lag_{lag}'] = y_train[-(lag - i)]
    
X_train, y_train = generate_lag_features_train(X_train, pd.DataFrame(y_train), lag_lst)[10*WINDOW_SZ+1:], y_train[10*WINDOW_SZ+1:]

Обучим модель

In [9]:
estimator_news = CatBoostRegressor(verbose=False, loss_function = 'MAPE').fit(np.array(X_train), y_train)
preds_news = estimator_news.predict(X_test)

Проведем такой же препроцессинг, только для данных без новстей

In [11]:
test_data = pd.read_csv('/content/drive/My Drive/data/test_table (2).csv',index_col = None)
preprocessed_test = preprocessing_pipline(test_data, use_text = False)

price_col = 'close'
X_train, X_test =  preprocessed_test[:-500], preprocessed_test[-500:]

y_train, y_test = volatility_estimation(X_train[price_col], 3), volatility_estimation(X_test[price_col], 3)

X_train, X_test = X_train[X_train.columns.difference([price_col])], X_test[X_test.columns.difference([price_col])]
X_train, X_test = X_train[:-(3+1)], X_test[:-(3+1)]
y_train, y_test = y_train[:-(3+1)], y_test[:-(3+1)]


WINDOW_SZ = 4
TEST_SZ = 4

X_train, X_test = create_date_features(X_train) , create_date_features(X_test)

lag_lst = [WINDOW_SZ+1, 2*WINDOW_SZ+1, 3*WINDOW_SZ+1, 5*WINDOW_SZ+1, 10*WINDOW_SZ+1]

X_test.reset_index(inplace = True, drop = True)
for lag in lag_lst:
  X_test[f'lag_{lag}'] = pd.DataFrame(y_test).shift(lag)
  for i in range(lag):
    X_test.at[i, f'lag_{lag}'] = y_train[-(lag - i)]
    
X_train, y_train = generate_lag_features_train(X_train, pd.DataFrame(y_train), lag_lst)[10*WINDOW_SZ+1:], y_train[10*WINDOW_SZ+1:]

<ipython-input-3-99eafe4fd354>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_clean = data_without_duple.append(data_concat)


Обучим модель

In [12]:
estimator = CatBoostRegressor(verbose=False, loss_function = 'MAPE').fit(np.array(X_train), y_train)
preds = estimator.predict(X_test)

Массивы для метрик

In [29]:
mape_news_lst = []
mape_without_news_lst = []

metr_news_lst = []
metr_without_news_lst = []

В прошлом ноутбуке выяснили, что для реальной задачи оценка качества по классическим метрикам содет привести к не очень хорошим результатам. Реализуем свою метрику, которая будет показавывать меру отклонения предсказанного максимума на 15 минут от действительного.

In [23]:
def metr(y_true, y_pred):
  true = np.amax(y_true)
  pred = np.amax(y_pred)
  return 100*(true - pred)/true

Проведем сбор метрик

In [30]:
for i in range(0, len(y_test), 3):
  y_true = y_test[i:i+3]
  y_pred_news = preds_news[i:i+3]
  y_pred_without_news =  preds[i:i+3]
  mape_news = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred_news)
  mape_without_news = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred_without_news)
  mape_news_lst.append(mape_news)
  mape_without_news_lst.append(mape_without_news)


  metr_news = metr(y_true=y_true, y_pred=y_pred_news)
  metr_without_news = metr(y_true=y_true, y_pred=y_pred_without_news)
  metr_news_lst.append(metr_news)
  metr_without_news_lst.append(metr_without_news)

Отлично! Собрали выборки. 

Мы имеем дело с зависимыми выборками, т.к. модели проходили обучение на обних и тех же данных. Применим критерий Стьюдента для зависимых выборок.

In [31]:
import scipy.stats as sps
sps.ttest_rel(mape_news_lst, mape_without_news_lst), sps.ttest_rel(metr_news_lst, metr_without_news_lst)

(TtestResult(statistic=1.052960463834486, pvalue=0.29389865357967726, df=165),
 TtestResult(statistic=-5.63212966000167, pvalue=7.515132173511468e-08, df=165))

Проинтерпретируем результаты тестов.

* По метрике **MAPE**. Видно, что по классической метрике не получилось отвергнуть гипотезу о том, что новости являются полезным фактором для предсказания. Это ожидаемо исходя из того, что можно было наблюдать в экспериментах до этого, хоть они и были поставлены на бОльшем горизонте предсказания.

* По классической метрике виден результат. Очень уверенно (с `p-value` $\approx 7^{-8}$) отвергаем гипотезу о незначимости новостей. Видно, что отклонение предсказанного максимума от реального на 5% меньше, если использовать новости в работе.

* Мы провели два теста на одних и тех же данных. Как следствие, у нас могла накопиться ошибка. Однако даже без применния продвинутых процедур МПГ можно видеть, что корректируя `p-value` даже с помощью метода Бонферрони результаты теста не меняются

## Выводы

* По результам теста понятно, что новости не должны влиять на качество предсказания чистой волатильности, где качество оцениваем по стандартной метрике.

* Несмотря на то, что есть гипотеза о том, что рынок "Итак все знает про новости", можно наблюдать, что их эффективно использовать в работе с тороговыми стратегиями. Это интуитивно понятно, учитывая, что на новости мы регаируем почти сразу с их выходом.

* По результатам теста смогли проверить гипотезы, которые выдвинули в ходе работы с данными и обучением моделей в разных конфигурациях